In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
#from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from vivit import R2Plus1D_ViViT
import gc
from einops import rearrange
from tqdm import tqdm
from ast_models_copy import ASTModel
import math
import copy





frame_nums=15
start_epoch = 0
test_flage=False
model_name =""
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()

In [2]:
logger.debug('============={}+Video Vision Transformer Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2025-09-25 15:04:48,662]::1914516053::DEBUG::=============+Video Vision Transformer Strat!=============


True
학습을 진행하는 기기: cuda


In [3]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            final_data_set.extend(pickle.load(output_file))
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            final_data_set.extend(pickle.load(output_file))
        return final_data_set
print(1)

1


In [4]:
save_model_file_path = './save_model_folder/{}_{}.{}'
print(1)

1


### add valid data path 

In [5]:
# validation_set_data=getdata("/data/qx/secwayofKrData/15Frames/test_set.dat","valid")
# print(1)
validation_set_data=getdata("Your Path","valid")
print(1)

1


### add train data path 

In [6]:
# train_set_data=getdata("/data/qx/secwayofKrData/15Frames/train_set.dat","train")
# print(1)
train_set_data=getdata("Your Path","train")
print(1)

1


In [7]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [8]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

# 데이터 텐서로 만들기

class ChalearnDataset(Dataset):  
    def __init__(self,audiodata,fullshot,textdata,tagdata, text_tokenizer, transform=None):
        self.audiodata=audiodata
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.textdata=textdata
        self.transform = transform  # 표준화 여부
        self.text_tokenizer = text_tokenizer
    
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        textdata = text_tokenizer(self.textdata[idx],
                                  return_tensors="pt",
                                     max_length=512,
                                     padding="max_length",
                                     truncation=True)
        textdata = {k: v.squeeze(0) for k, v in textdata.items()}  
        
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces/120)

        out = {"audio": audiodata,
              "image": fullshot,
              "text_input_ids": textdata["input_ids"],
              "text_attn_mask": textdata["attention_mask"],
              "labels": big_five_sorces}

        
        return out
print(1)

1


In [9]:
num_workerssz = 20
lr = 3e-05
epochs = 120
print(1)

1


### data 구성[[audio],[video],[transcript],[labels]]
### 저희측에서 추출된 transcript는 하나의 시나리오 중에서 이 사람 말하는 모두 내용입니다

In [10]:
text_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

train_set_dataset = ChalearnDataset(audiodata= train_set_data[0], fullshot = train_set_data[1], textdata= train_set_data[2], 
                                 tagdata=train_set_data[3], transform=transform, text_tokenizer = text_tokenizer)
val_set_dataset = ChalearnDataset(audiodata= validation_set_data[0], fullshot = validation_set_data[1], textdata= validation_set_data[2],
                               tagdata=validation_set_data[3],transform=transform, text_tokenizer = text_tokenizer)


train_dataloader = DataLoader(dataset=train_set_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(dataset=val_set_dataset, batch_size=8, shuffle=True)
max_value=0
print(1)

1


In [11]:
train_set_dataset.__getitem__(10)['labels']

tensor([[0.7917],
        [0.7417],
        [0.5667],
        [0.5583],
        [0.7250]])

In [12]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads=12, hidden_size=768, d_out=0.1, position_embedding_type=None):
        super().__init__()

        self.num_attention_heads = num_heads
        self.attention_head_size = int(hidden_size / num_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(hidden_size, self.all_head_size)
        self.key = nn.Linear(hidden_size, self.all_head_size)
        self.value = nn.Linear(hidden_size, self.all_head_size)
        self.linear = nn.Linear(hidden_size, hidden_size)
        self.hidden_size = hidden_size

        self.dropout = nn.Dropout(d_out)

    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.shape[0]
        x = x.view(batch_size, -1, self.num_attention_heads, self.attention_head_size)
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, value, mask=None):        
        batch_size = query.shape[0]
        mixed_query_layer = self.query(query)

        # [batch_size, n_heads, query_len, head_dim]
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(self.key(key))
        value_layer = self.transpose_for_scores(self.value(value))

        # Take the dot product between "query" and "key" to get the raw attention scores.
        # [batch_size, n_heads, query_len, key_len]
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)


        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e10)
        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # [batch_size, n_heads, query_len, head_dim]
        context_layer = torch.matmul(attention_probs, value_layer)

        # [batch_size, query_len, n_heads, head_dim]
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()

        # [batch_size, query_len, hid_dim]
        context_layer = context_layer.view(batch_size, -1, self.hidden_size)

        out = self.linear(context_layer)

        return out, attention_probs

class AttentionBlock(nn.Module):
    def __init__(self, hidden_size=768, num_heads=12, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(num_heads=num_heads, hidden_size=hidden_size, d_out=dropout)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 4),
            nn.GELU(),
            nn.Linear(hidden_size * 4, hidden_size),
            nn.Dropout(dropout)
        )
        self.norm2 = nn.LayerNorm(hidden_size)

    def forward(self, query, key, value, mask=None):
     #   query = self.norm1(query)
        out, attn_probs = self.attn(query, key, value, mask)
     #   out = query + self.dropout(out)  # Residual

      #  out = self.norm2(out)
      #  out = out + self.ffn(out)
        
        return out, attn_probs


class MultiModalAttentionModel(nn.Module):
    def __init__(self, text_model, image_model, audio_model, num_labels=5, projection_dim=768, d_out=0.1, ):
        super().__init__()
        self.projection_dim = projection_dim
        self.num_labels = num_labels
        
        self.text_model = text_model
        self.image_model = image_model
        self.audio_model = audio_model

        # Freeze
        for name, param in self.text_model.named_parameters():
            param.requires_grad = False
        for name, param in self.image_model.named_parameters():
            param.requires_grad = False
        for name, param in self.audio_model.named_parameters():
            param.requires_grad = False

        
        self.mh_attention_1 = nn.ModuleList([AttentionBlock() for _ in range(6)])  # N-CA Blocks
        self.mh_attention_2 = nn.ModuleList([AttentionBlock() for _ in range(6)])
        self.mh_attention_3 = nn.ModuleList([AttentionBlock() for _ in range(6)])
        self.mh_attention_4 = nn.ModuleList([AttentionBlock() for _ in range(6)])
        self.mh_attention_5 = nn.ModuleList([AttentionBlock() for _ in range(6)])
        self.mh_attention_6 = nn.ModuleList([AttentionBlock() for _ in range(6)])

        # self.mh_attention_1 = MultiHeadAttention()
        # self.mh_attention_2 = MultiHeadAttention()
        # self.mh_attention_3 = MultiHeadAttention()

        
        self.dense = nn.Linear(self.projection_dim * 2, self.projection_dim * 2)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(d_out)
        self.out_proj = nn.Linear(self.projection_dim * 2, self.num_labels)

    def forward(
        self,
        audio,
        image,
        text_input_ids,
        text_attn_mask,
        labels = None,
        position_ids = None,
        head_mask = None,
        inputs_embeds = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None,
    ):

        # attention 모듈 제외하고 모두 freeze
        with torch.no_grad():
            text_outputs = self.text_model(
                input_ids = text_input_ids,
                attention_mask = text_attn_mask,
            )

            image_outputs = self.image_model(image.permute(0,4,1,2,3).contiguous()).unsqueeze(1)
 
            audio_outputs = self.audio_model(audio.squeeze()).unsqueeze(1)

        text_embeds = text_outputs[0][:, 0, :].unsqueeze(1)
        text_mask = text_attn_mask.unsqueeze(1).unsqueeze(2)  # [B, 1, 1, text_len]
        # text_embeds = text_outputs[0][:,0,:]  # [batch_size, 768(hidden)]

        # print("Text model out: ", text_embeds.shape)
        # print("Image model out:", image_outputs.shape)
        # print("Audio model out:", audio_outputs.shape)

        # ca_a, ca_a_prob = self.mh_attention_1(audio_outputs, image_outputs, image_outputs)
        # ca_at, ca_at_prob = self.mh_attention_2(audio_outputs, text_embeds, text_embeds)
        # ca_t, ca_t_prob = self.mh_attention_1(text_embeds, image_outputs, image_outputs)
        # ca_ta, ca_ta_prob = self.mh_attention_2(text_embeds, audio_outputs, audio_outputs)
        # ca_v, ca_v_prob = self.mh_attention_1(image_outputs, text_embeds, text_embeds)
        # ca_va, ca_va_prob = self.mh_attention_2(image_outputs, audio_outputs, audio_outputs)

        image_outputs_cp = image_outputs.clone()
        image_outputs_cp2 = image_outputs.clone()
        text_outputs_cp = text_embeds.clone()
        text_outputs_cp2 = text_embeds.clone()
        audio_outputs_cp = audio_outputs.clone()

        ## N CA Layers
        #### Image Key
        for idx, layer in enumerate(self.mh_attention_1):
           # print(idx, "th: ", image_outputs.shape)
            # out, _ = layer(image_outputs, text_embeds, text_embeds)
            # image_outputs = out
            out, _ = layer(image_outputs, text_embeds, text_embeds)
            image_outputs = out

        for layer2 in self.mh_attention_2:
            out2, _ = layer2(image_outputs_cp, audio_outputs, audio_outputs)
            image_outputs_cp = out2

        # #### Text key
        # for layer3 in self.mh_attention_3:
        #     out3, _ = layer3(text_embeds, audio_outputs, audio_outputs)
        #     text_embeds = out3

        # for layer4 in self.mh_attention_4:
        #     out4, _ = layer4(text_outputs_cp, image_outputs_cp2, image_outputs_cp2)
        #     text_outputs_cp = out4

        # #### Audio key
        # for layer5 in self.mh_attention_5:
        #     out5, _ = layer5(audio_outputs, image_outputs_cp2, image_outputs_cp2)
        #     audio_outputs = out5

        # for layer6 in self.mh_attention_6:
        #     out6, _ = layer6(audio_outputs_cp, text_outputs_cp2, text_outputs_cp2)
        #     audio_outputs_cp = out6

        # print(out.shape)
        # print(out2.shape)



        #################################  Don't forget  ######################
        concated_input = torch.concat([out.squeeze(1), out2.squeeze(1),
                                      # out3.squeeze(1), out4.squeeze(1),
                                      # out5.squeeze(1), out6.squeeze(1)
                                      ], dim=-1)  # [Batch_size, hidden * n_CA] 
        x = concated_input



        x = self.dropout(x)
        x = self.dense(x)
        x = self.act(x)  # GELU
        x = self.dropout(x)
        logits = self.out_proj(x)  # [Batch_Size x 5]

        loss = None

        if labels != None:
            loss_fct = nn.L1Loss()
    
           # loss_fct = loss_fct(logits_O.view(-1, self.num_labels), labels[0])
            loss = loss_fct(logits.squeeze(), labels.squeeze())

        return loss, logits

In [13]:
#num_frames=frame_nums,batchsz = batchsz
image_model=R2Plus1D_ViViT(2, # t
                     14,  # h
                     14,  # w
                     2,   # patch_t
                     2,   # patch_h
                     2,   # patch_w
                     5,   # num_classes
                     768,  # dim
                     6,   # depth
                     10,  # heads
                     8,   #  mlp_dim
                     model=3) 

text_model = AutoModel.from_pretrained('/home/egg2018037024/Jupyter_Home/Multi-modal_personality/koRoBERTa_base-checkpoint16000')
audio_model = ASTModel(input_tdim=1319, input_fdim=24, audioset_pretrain=True) # audio_split_samples, audio_length

whole_model = MultiModalAttentionModel(text_model= text_model, 
                                       image_model = image_model, 
                                       audio_model = audio_model,
                                      num_labels = 5)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/egg2018037024/Jupyter_Home/Multi-modal_personality/koRoBERTa_base-checkpoint16000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
audioset_pretrain == True
[PE] Not use imagenet pretrain
frequncey stride=10, time stride=10
number of patches=599


In [14]:
sum(p.numel() for p in whole_model.parameters() if p.requires_grad)

257531909

In [15]:
from torchinfo import summary

summary(whole_model)

Layer (type:depth-idx)                                            Param #
MultiModalAttentionModel                                          --
├─RobertaModel: 1-1                                               --
│    └─RobertaEmbeddings: 2-1                                     --
│    │    └─Embedding: 3-1                                        (24,576,000)
│    │    └─Embedding: 3-2                                        (394,752)
│    │    └─Embedding: 3-3                                        (768)
│    │    └─LayerNorm: 3-4                                        (1,536)
│    │    └─Dropout: 3-5                                          --
│    └─RobertaEncoder: 2-2                                        --
│    │    └─ModuleList: 3-6                                       (85,054,464)
│    └─RobertaPooler: 2-3                                         --
│    │    └─Linear: 3-7                                           (590,592)
│    │    └─Tanh: 3-8                                   

### Trainer

In [16]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import multilabel_confusion_matrix
from transformers import EvalPrediction
from sklearn.metrics import mean_absolute_error
import torch

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    MAE = 1 - mean_absolute_error(labels.squeeze(), preds)
    
    
    return {
        '1 - MAE' : MAE
    }

In [17]:
from transformers import TrainingArguments, Trainer, TrainerCallback, DefaultDataCollator

training_args = TrainingArguments(
    output_dir="/home/user/10TB/personalityAI/multimodal/6CA_Ikey",
    logging_dir= "/home/user/10TB/personalityAI/multimodal/6CA_Ikey_log",
    num_train_epochs=10,
    learning_rate = 3e-5, 
   # max_steps=1000,
    per_device_train_batch_size=32,
#    gradient_accumulation_steps = 16,
    per_device_eval_batch_size = 16,
#    eval_accumulation_steps = 32,
    logging_strategy = "epoch",
    save_strategy = "epoch",
    lr_scheduler_type = "linear",
 #   dataloader_num_workers = 12,
    warmup_ratio = 0.1,
#    weight_decay=0.01,
    eval_strategy='epoch',
)

trainer = Trainer(
    model=whole_model,
    args=training_args,
    train_dataset=train_set_dataset,
    eval_dataset=val_set_dataset,
   # data_collator=DefaultDataCollator(return_tensors = "pt"),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()   #0.912889

### Test

In [25]:
# train_set_data=getdata("/data/qx/secwayofKrData/15Frames/train_set.dat","train")
# print(1)
test_set_data=getdata("Your Path","valid")
print(1)

1


In [26]:
test_set_dataset = ChalearnDataset(audiodata= test_set_data[0], fullshot = test_set_data[1], textdata= test_set_data[2],
                               tagdata=test_set_data[3],transform=transform, text_tokenizer = text_tokenizer)

In [27]:
 def test_compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    
    labels = labels.squeeze().T
    preds = preds.T
    
    MAE = []
    
    for i in range(5):
        MAE.append(1 - mean_absolute_error(labels[i], preds[i]))
    
    
    return {
        'Avg' : np.mean(MAE),
        '1 - MAE' : MAE
    }

In [28]:
from safetensors.torch import load_model

test_model = MultiModalAttentionModel(text_model= text_model, 
                                       image_model = image_model, 
                                       audio_model = audio_model,
                                      num_labels = 5)


load_model(test_model, "/home/user/10TB/personalityAI/multimodal/6CA_Ikey/checkpoint-238/model.safetensors")

(set(), [])

In [29]:
from transformers import TrainingArguments, Trainer, TrainerCallback, DefaultDataCollator

test_training_args = TrainingArguments(
    output_dir="/home/user/10TB/personalityAI/multimodal/6CA_Ikey",
    logging_dir= "/home/user/10TB/personalityAI/multimodal/6CA_Ikey_log",
    num_train_epochs=10,
    learning_rate = 3e-5,  
   # max_steps=1000,
    per_device_train_batch_size=32,
#    gradient_accumulation_steps = 16,
    per_device_eval_batch_size = 32,
#    eval_accumulation_steps = 32,
    logging_strategy = "epoch",
    lr_scheduler_type = "linear",
 #   dataloader_num_workers = 12,
    warmup_ratio = 0.1,
#    weight_decay=0.01,
)

eval_trainer = Trainer(
    model=test_model,
    args=test_training_args,
    data_collator=DefaultDataCollator(return_tensors = "pt"),
    compute_metrics=test_compute_metrics,
)

In [30]:
eval_trainer.evaluate(test_set_dataset) 

/home/egg2018037024/venv310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.09143953770399094,
 'eval_Avg': 0.9085604593157768,
 'eval_1 - MAE': [0.9117942899465561,
  0.9110761284828186,
  0.8928183913230896,
  0.9298798218369484,
  0.8972336649894714],
 'eval_runtime': 716.0798,
 'eval_samples_per_second': 7.105,
 'eval_steps_per_second': 0.056}